## Lab 4: 本番環境へのデプロイ - AgentCore Runtime とオブザーバビリティの使用

### 概要

Lab 3 では、安全な認証を備えた AgentCore Gateway を通じてツールを集中化することで、カスタマーサポートエージェントをスケールしました。次に、包括的なオブザーバビリティを備えた AgentCore Runtime にエージェントをデプロイして、本番環境への移行を完了します。これにより、プロトタイプを実世界のトラフィックを処理できる本番環境対応システムに変換し、完全な監視と自動スケーリングを実現します。

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) は、フレームワーク、プロトコル、モデルの選択に関係なく、組織が AI エージェントを本番環境でデプロイおよびスケールできるようにする、安全でフルマネージドなランタイムです。エンタープライズグレードの信頼性、自動スケーリング、包括的な監視機能を提供します。

**ワークショップの流れ:**

- **Lab 1（完了）:** エージェントプロトタイプの作成 - 機能するカスタマーサポートエージェントを構築
- **Lab 2（完了）:** Memory による強化 - 会話コンテキストとパーソナライゼーションを追加
- **Lab 3（完了）:** Gateway と Identity によるスケーリング - エージェント間でツールを安全に共有
- **Lab 4（現在）:** 本番環境へのデプロイ - AgentCore Runtime を使用してオブザーバビリティを実現
- **Lab 5:** エージェントパフォーマンスの評価 - オンライン評価で品質を監視
- **Lab 6:** ユーザーインターフェースの構築 - 顧客向けアプリケーションを作成

### AgentCore Runtime と本番環境デプロイが重要な理由

現在の状態（Lab 1-3）: エージェントは集中化されたツールでローカルで実行されていますが、本番環境での課題に直面しています：

- エージェントが単一のセッションでローカルに実行される
- 包括的な監視やデバッグ機能がない
- 複数の同時ユーザーを確実に処理できない

このLabの後は、以下を備えた本番環境対応のエージェントインフラストラクチャを持つことになります：

- 可変需要に対応するサーバーレス自動スケーリング
- トレース、メトリクス、ロギングによる包括的なオブザーバビリティ
- 自動エラーリカバリを備えたエンタープライズ信頼性
- 適切なアクセス制御を備えた安全なデプロイ
- AWS コンソールと API を通じた簡単な管理と、実世界の本番ワークロードのサポート


### AgentCore Observability による包括的なオブザーバビリティの追加

さらに、AgentCore Runtime は [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) とシームレスに統合され、本番環境でのエージェントの動作を完全に可視化します。AgentCore Observability は、エージェントのインタラクション、ツールの使用状況、Memory アクセスパターンからトレース、メトリクス、ログを自動的にキャプチャします。このLabでは、AgentCore Runtime が CloudWatch GenAI Observability と統合して、包括的な監視とデバッグ機能を提供する方法を見ていきます。

リクエストのトレーシングでは、AgentCore Observability はツールの呼び出し、Memory の取得、モデルのインタラクションを含む完全な会話フローをキャプチャします。パフォーマンス監視では、応答時間、成功率、リソース使用率を追跡して、エージェントのパフォーマンスを最適化するのに役立ちます。

オブザーバビリティのフロー中、AgentCore Runtime はエージェントコードを自動的にインストルメント化し、テレメトリデータを CloudWatch に送信します。その後、CloudWatch ダッシュボードと GenAI Observability 機能を使用して、パターンを分析し、ボトルネックを特定し、リアルタイムで問題をトラブルシューティングできます。

### Lab 4 のアーキテクチャ
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*エージェントは CloudWatch を通じた完全なオブザーバビリティを備えた AgentCore Runtime で実行され、自動スケーリングと包括的な監視で本番トラフィックを処理します。以前のLabからの Memory と Gateway の統合は、本番環境で完全に機能し続けます。*

### 主な機能

- **サーバーレスエージェントデプロイ:** 最小限のコード変更で AgentCore Runtime を使用して、ローカルエージェントをスケーラブルな本番サービスに変換
- **包括的なオブザーバビリティ:** CloudWatch GenAI Observability を通じた完全なリクエストトレーシング、パフォーマンスメトリクス、デバッグ機能

### 前提条件

- Python 3.12+
- 適切な権限を持つ AWS アカウント
- Docker、Finch または Podman がインストールされ実行中
- Amazon Bedrock AgentCore SDK
- Strands Agents フレームワーク
- **Lab 3 の完了:** このLabは Lab 3（AgentCore Gateway）の上に構築されています。このLabを実行する前に、[lab-03-agentcore-gateway](lab-03-agentcore-gateway.ipynb) を実行して Gateway をプロビジョニングする必要があります。

**注意**: AgentCore Observability のトレースを CloudWatch で表示するには、[CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) を有効にする必要があります。

### ステップ 1: 必要なライブラリのインポート

In [ ]:
# Import required libraries
import boto3
from lab_helpers.utils import get_ssm_parameter
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.constants import StrategyType


boto_session = boto3.Session()
REGION = boto_session.region_name


memory_name = "CustomerSupportMemory"
memory_manager = MemoryManager(region_name=REGION)
memory = memory_manager.get_or_create_memory( # Just in case the memory lab wasn't executed
    name=memory_name,
    strategies=[
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "CustomerPreferences",
                        "description": "Captures customer preferences and behavior",
                        "namespaces": ["support/customer/{actorId}/preferences"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "CustomerSupportSemantic",
                        "description": "Stores facts from conversations",
                        "namespaces": ["support/customer/{actorId}/semantic"],
                    }
                },
            ]
)
memory_id = memory["id"]

### ステップ 2: AgentCore Runtime 用のエージェントの準備

#### Runtime 対応エージェントの作成

まず、以前のローカルエージェント実装内で、Python SDK を介して必要な AgentCore Runtime コンポーネントを定義しましょう。

以下の #### AGENTCORE RUNTIME - LINE 1 #### コメントを観察して、関連するデプロイコードが追加された場所を確認してください。Runtime 対応エージェントを準備する4つの行があります：

1. `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
2. `app = BedrockAgentCoreApp()` で App を初期化
3. 呼び出し関数を `@app.entrypoint` でデコレート
4. `app.run()` で AgentCore Runtime に実行を制御させる

##### 実装の主要な詳細：

Runtime 対応エージェントは、ペイロードからユーザープロンプトを抽出し、context.request_headers.get('Authorization', '') を介してリクエストヘッダーから JWT トークンを抽出するエントリポイント関数を使用します。認可トークンは、MCP クライアントヘッダーに渡すことで AgentCore Gateway に直接伝播されます：headers={"Authorization": auth_header}。実装には、認証が欠落している場合のエラー処理が含まれ、以前のLabからのすべての Memory とツール機能を維持しながら、同期エージェント呼び出しからプレーンテキストレスポンスを返します。

In [ ]:
%%writefile ./lab_helpers/lab4_runtime.py
import os
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import requests
import boto3
from strands.models import BedrockModel
from lab_helpers.utils import get_ssm_parameter
from lab_helpers.lab1_strands_agent import (
    get_return_policy,
    get_product_info,
    get_technical_support,
    SYSTEM_PROMPT,
    MODEL_ID,
)

from lab_helpers.lab2_memory import (
    ACTOR_ID,
    SESSION_ID,
)
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig, RetrievalConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

# Initialize boto3 client
sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

# Lab1 import: Create the Bedrock model
model = BedrockModel(model_id=MODEL_ID)

# Lab2 import: Memory
memory_id = os.environ.get("MEMORY_ID")
if not memory_id:
    raise Exception("環境変数 MEMORY_ID が必要です")

memory_config = AgentCoreMemoryConfig(
        memory_id=memory_id,
        session_id=str(SESSION_ID),
        actor_id=ACTOR_ID,
        retrieval_config={
            "support/customer/{actorId}/semantic": RetrievalConfig(top_k=3, relevance_score=0.2),
            "support/customer/{actorId}/preferences": RetrievalConfig(top_k=3, relevance_score=0.2)
        }
    )

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####

@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
async def invoke(payload, context=None):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")

    # Access request headers - handle None case
    request_headers = context.request_headers or {}

    # Get Client JWT token
    auth_header = request_headers.get('Authorization', '')

    print(f"Authorization ヘッダー: {auth_header}")
    # Get Gateway ID
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    
    # Initialize Bedrock AgentCore Control client
    gateway_client = boto3.client(
        "bedrock-agentcore-control",
        region_name=REGION,
    )
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)

    # Get gateway url
    gateway_url = gateway_response['gatewayUrl']

    # Create MCP client and agent within context manager if JWT token available
    if gateway_url and auth_header:
        try:
                mcp_client = MCPClient(lambda: streamablehttp_client(
                    url=gateway_url,
                    headers={"Authorization": auth_header}  
                ))
                
                with mcp_client:
                    tools = (
                        [
                            get_product_info,
                            get_return_policy,
                            get_technical_support
                        ]
                        + mcp_client.list_tools_sync()
                    )
                    
                    # Create the agent with all customer support tools
                    agent = Agent(
                        model=model,
                        tools=tools,
                        system_prompt=SYSTEM_PROMPT,
                        session_manager=AgentCoreMemorySessionManager(memory_config, REGION),
                    )
                    # Invoke the agent
                    response = agent(user_input)
                    return response.message["content"][0]["text"]
        except Exception as e:
                print(f"MCP クライアントエラー: {str(e)}")
                return f"エラー: {str(e)}"
    else:
        return "エラー: Gateway URL または Authorization ヘッダーがありません"

if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

#### 裏側で何が起こるのか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます：

- ポート 8080 でリッスンする HTTP サーバーを作成
- リクエスト処理に必要な `/invocations` エンドポイントを実装
- ヘルスチェック用の `/ping` エンドポイントを実装
- 適切なコンテンツタイプとレスポンス形式を処理
- AWS 標準に従ったエラー処理を管理

### ステップ 3: AgentCore Runtime へのデプロイ

次に、[AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit) を使用して、エージェントを AgentCore Runtime にデプロイしましょう。

#### 安全な Runtime デプロイの設定（AgentCore Runtime + AgentCore Identity）

まず、スターターツールキットを使用して、エントリポイント、作成する実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、Amazon Cognito ユーザープールを使用して Identity 認可を設定し、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left"> 
    <img src="images/configure.png" width="75%"/> 
</div>

**注意**: Cognito access_token は2時間のみ有効です。access_token が期限切れになった場合は、`reauthenticate_user` メソッドを使用して別の access_token を発行できます。

In [ ]:
from lab_helpers.utils import get_or_create_cognito_pool

access_token = get_or_create_cognito_pool(refresh_token=True)
print(f"アクセストークン: {access_token['bearer_token']}")

#### AgentCore Runtime 設定の概要：

以下のコードは、スターターツールキットを使用して AgentCore Runtime デプロイを設定します。Runtime 用の実行ロールを作成し、エージェントエントリポイントファイル（lab_helpers/lab4_runtime.py）でデプロイを設定し、自動 ECR リポジトリ作成を有効にし、Cognito を使用した JWT ベースの認証をセットアップします。設定では、SSM パラメータから取得した許可されたクライアント ID とディスカバリ URL を指定し、本番エージェントデプロイのための安全なアクセス制御を確立します。このステップでは、コンテナ化されたデプロイに必要な Dockerfile と .bedrock_agentcore.yaml 設定ファイルが自動的に生成されます。

**Runtime ヘッダー設定**: 以下のコードは、デプロイされた AgentCore Runtime のカスタムヘッダー許可リストを設定します。エージェント ARN から Runtime ID を抽出し、既存の設定を保持するために現在の Runtime 設定を取得してから、Authorization ヘッダー（OAuth トークン伝播に必要）とカスタムヘッダーを含むリクエストヘッダー許可リストで Runtime を更新します。これにより、JWT トークンやその他の必要なヘッダーがクライアントリクエストからエージェント Runtime コードに適切に転送されます。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [
                get_ssm_parameter("/app/customersupport/agentcore/client_id")
            ],
            "discoveryUrl": get_ssm_parameter(
                "/app/customersupport/agentcore/cognito_discovery_url"
            ),
        }
    },
    # Add custom header allowlist for Authorization and custom headers
    request_header_configuration={
        "requestHeaderAllowlist": [
            "Authorization",  # Required for OAuth propogation
            "X-Amzn-Bedrock-AgentCore-Runtime-Custom-H1",  # Custom header
        ]
    },
)

print("設定完了:", response)

#### エージェントの起動

次に、エージェントを AgentCore Runtime に起動しましょう。これにより、AWS CodeBuild パイプライン、Amazon ECR リポジトリ、および AgentCore Runtime コンポーネントが作成されます。

<div style="text-align:left"> 
    <img src="images/launch.png" width="100%"/> 
</div>

*注意: 同じ名前のエージェントが既に存在する場合、このステップは失敗する可能性があります。既存の Runtime を上書きする場合は、代わりにこれを使用してください：*

``` launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)```

In [ ]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch(env_vars={"MEMORY_ID": memory_id})
print("起動完了:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/customersupport/agentcore/runtime_arn", launch_result.agent_arn
)

#### デプロイステータスの確認

デプロイが完了するまで待ちましょう：

In [ ]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"デプロイ待機中... 現在のステータス: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"最終ステータス: {status}")

### ステップ 4: デプロイされたエージェントの呼び出し

エージェントがデプロイされて準備ができたので、いくつかのクエリでテストしましょう。適切な認可トークンタイプでエージェントを呼び出します。この場合は Cognito アクセストークンになります。上のセルからアクセストークンをコピーしてください。

<div style="text-align:left"> 
    <img src="images/invoke.png" width="100%"/> 
</div>

#### AgentCore Starter Toolkit の使用

AgentCore Starter Toolkit を使用してエージェントが動作することを検証できます。スターターツールキットは、エージェントにクエリするためのセッション ID を自動的に作成できます。または、呼び出し時にセッション ID をパラメータとして渡すこともできます。デモンストレーションの目的で、独自のセッション ID を作成します。

In [ ]:
# Initialize the AgentCore Control client
client = boto3.client("bedrock-agentcore-control")

# Extract runtime ID from the ARN (format: arn:aws:bedrock-agentcore:region:account:runtime/runtime-id)
runtime_id = launch_result.agent_arn.split(":")[-1].split("/")[-1]

print(f"ランタイム ID: {runtime_id}")

In [ ]:
import uuid
from IPython.display import display, Markdown

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

# Test different customer support scenarios
user_query = "List all of your tools"

response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id),
)

display(Markdown(response["response"].replace('\\n', '\n')))

#### セッション継続性を持つエージェントの呼び出し

AgentCore Runtime を使用しているため、同じセッション ID で会話を簡単に継続できます。

In [ ]:
user_query = "Tell me detailed information about the technical documentation on installing a new CPU"
response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id),
)
display(Markdown(response["response"].replace('\\n', '\n')))

#### 新しいユーザーでエージェントを呼び出す
以下の例では、2番目のクエリで iPhone デバイスについて言及していませんが、エージェントはまだそのコンテキストを持っています。これは AgentCore Runtime のセッション継続性によるものです。新しいユーザーの場合、エージェントはコンテキストを知りません。

In [ ]:
# Creating a new session ID for demonstrating new customer
session_id2 = uuid.uuid4()

user_query = "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333."
response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id2),
)
display(Markdown(response["response"].replace('\\n', '\n')))

この場合、エージェントはもうコンテキストを持っておらず、より多くの情報が必要です。

そして、基盤となるすべてのインフラストラクチャを管理する必要なく、エージェント用の安全でスケーラブルなエンドポイントを持つために必要なのはこれだけです！

### ステップ 5: AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) は、Amazon OpenTelemetry Python Instrumentation と Amazon CloudWatch GenAI Observability を使用して、AI エージェントの監視とトレーシング機能を提供します。

#### エージェント

デフォルトの AgentCore Runtime 設定では、**AgentCore Observability** を使用して CloudWatch にエージェントのトレースをログ記録できます。これらのトレースは AWS CloudWatch GenAI Observability ダッシュボードで確認できます。CloudWatch &rarr; GenAI Observability &rarr; Bedrock AgentCore に移動します。

![CloudWatch でのエージェント概要](images/observability_agents.png)

#### セッション

セッションビューには、アカウント内のすべてのエージェントに関連付けられたすべてのセッションのリストが表示されます。

![セッション](images/sessions_lab5_observability.png)

#### トレース

トレースビューには、アカウント内のエージェントからのすべてのトレースがリストされます。トレースを操作するには：

- 特定のトレースを検索するには「Filter traces」を選択します
- 結果を整理するには列名でソートします
- 「Actions」で、ログとスパンデータ全体をクエリして検索を絞り込むには「Logs Insights」を選択するか、「Export selected traces」を選択してエクスポートします

![トレース](images/traces_lab4_observability.png)

### おめでとうございます！

**Lab 4: 本番環境へのデプロイ - AgentCore Runtime とオブザーバビリティの使用**を正常に完了しました！

達成したこと：

##### 本番環境対応デプロイ：

- 最小限のコード変更（4行のみ追加）でエージェントを本番環境用に準備
- 異なる顧客間での適切なセッション分離を検証
- セッション継続性 + Memory の永続性とセッションごとのコンテキスト認識を確認

##### エンタープライズグレードのセキュリティと Identity：

- JWT トークンを使用した Cognito 統合による安全な認証を実装
- 本番ワークロード用の適切な IAM ロールと実行権限を設定
- 安全なエージェント呼び出しのための Identity ベースのアクセス制御を確立

##### 包括的なオブザーバビリティ：

- すべての顧客セッションにわたる完全なリクエストトレーシングのための AgentCore Observability を有効化
- CloudWatch GenAI Observability ダッシュボード監視を設定

##### 現在の制限事項（次のLabで修正します！）：

- **開発者向けのインタラクション** - SDK/API 呼び出しでエージェントにアクセスできるが、ユーザーフレンドリーなウェブインターフェースがない
- **手動セッション管理** - 直感的なユーザー体験ではなく、プログラムによるセッション作成が必要

##### 次のステップ [Lab 5: エージェントパフォーマンスの評価 →](lab-05-agentcore-evals.ipynb)
Lab 5 では、本番エージェントが高いパフォーマンス基準を維持できるよう、AgentCore Evaluations による継続的な品質監視を設定します！